In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import random
import torchvision
from tqdm import tqdm
import os
from PIL import Image
from glob import glob
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# setting seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)
random.seed(1)

In [4]:
# set data path
train_dir_path = "./2021-ai-w10-p2/images/images"
test_dir_path = "./2021-ai-w10-p2/test_data/test_data"
submission_csv_path = "./2021-ai-w10-p2/format.csv"

In [27]:
# define custom dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,dir_path,split,transform = None):
        self.split = split
        self.tranform = transform
        if self.split == "train":
            self.data_path = glob(os.path.join(dir_path,"*","*"))
            self.data_path = [x.replace("\\",'/') for x in self.data_path]
            self.label = [int(x.split('/')[-2]) for x in self.data_path]
        else:
            self.data_path = glob(os.path.join(dir_path,"*"))
            # 정렬을 위해
            self.data_path = [dir_path+"/"+str(x)+".png" for x in range(len(self.data_path))]
        self.len = len(self.data_path)
    
    def __len__(self):
        return self.len
    
    def __getitem__(self,index):
        img = Image.open(self.data_path[index])
        img = transform(img)
        if self.split == "train":
            return img, torch.LongTensor([self.label[index]])
        else:
            return img

In [6]:
# use transforms
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [7]:
train_dataset = MyDataset(train_dir_path,"train",transform)

In [8]:
x,y = train_dataset.__getitem__(33334)

In [9]:
model = torchvision.models.googlenet(pretrained=True)

# freezing
# for param in model.parameters():
#     param.requires_grad = False

model.fc = torch.nn.Linear(in_features=1024, out_features=10, bias=True).to(device)

torch.nn.init.xavier_normal_(model.fc.weight)

model = model.to(device)
# print(model)

In [10]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 50
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
epochs = 10
loss = torch.nn.CrossEntropyLoss()

In [11]:
def train_model(model, data_loader, optim):
    sum_cost = 0
    sum_acc = 0
    for x, y in tqdm(data_loader):
        x = x.to(device)
        y = y.to(device).view(-1)
        model.train()
        output = model(x)
        cost = loss(output, y)
        optim.zero_grad()
        cost.backward()
        optim.step()
        sum_cost += cost.item()
        model.eval()
        predict = model(x)
        predict = torch.argmax(predict,dim=1)
        acc = predict == y
        sum_acc += acc.sum().item()
    return sum_cost,sum_acc

In [12]:
# learning
for epoch in range(epochs + 1):
    cost, acc = train_model(model, train_data_loader, optim)
    cost = cost / len(train_data_loader.dataset)
    acc = acc/ len(train_data_loader.dataset)  
    print(epoch, cost,acc)

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]c:\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:10<00:00,  7.67it/s]


0 0.010512616210430861 0.86682


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:09<00:00,  7.72it/s]


1 0.00637860158264637 0.935


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:09<00:00,  7.72it/s]


...

8 0.0018390367479808629 0.99256


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:09<00:00,  7.75it/s]


9 0.0016101649667532184 0.99484


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:09<00:00,  7.75it/s]

10 0.0014221705814055168 0.99558


In [34]:
with torch.no_grad():
    model.eval()
    acc = 0
    for x, y in tqdm(train_data_loader):
        x = x.to(device)
        y = y.to(device).view(-1)
        predict = model(x)
        predict = torch.argmax(predict, dim=1)
        acc += (y == predict).sum().item()
print(acc/len(train_data_loader.dataset))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:19<00:00, 12.55it/s]

0.98262


In [31]:
# set test data on Tensor and Dataset
test_dataset = MyDataset(test_dir_path,"test",transform)
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [32]:
def test_model(model, data_loader):
    predicts = []
    for x in tqdm(data_loader):
        x = x.to(device)
        predict = model(x)
        predict = torch.argmax(predict, dim=1)
        predicts += np.array(predict.cpu().detach()).tolist()
    return np.array(predicts)

In [33]:
# submit
submission = pd.read_csv(submission_csv_path)
with torch.no_grad():
    model.eval()
    predict = test_model(model, test_data_loader)
    submission['label'] = predict
    print(submission)
submission.to_csv("submission.csv", index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:15<00:00, 12.84it/s]

         id  label
0         1      3
1         2      8
2         3      8
3         4      0
4         5      6
...     ...    ...
9995   9996      8
9996   9997      3
9997   9998      5
9998   9999      1
9999  10000      7

[10000 rows x 2 columns]
